# Situp Counter using MediaPipe

This notebook implements a machine learning-based situp counter using MediaPipe Pose estimation. The system:
- Detects body landmarks using MediaPipe
- Tracks hip and shoulder angles to identify situp movements
- Counts completed situps in a 30-second window
- Saves the trained model for future use

## 1. Install Required Libraries

In [1]:
!pip install mediapipe opencv-python numpy scikit-learn joblib 

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/b7/79/b77808f8195f229ef0c15875540dfdd36724748a4b3de53d993f23336839/mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/fa/80/eb88edc2e2b11cd2dd2e56f1c80b5784d11d6e6b7f04a1145df64df40065/opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/3d/a8/566578b10d8d0e9955b1b6cd5db4e9d4592dd0026a941ff7994cedda030a/numpy-2.3.4-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.9 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 60.9/60.9 kB 1.6 MB/s eta 0:00:00
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/c6/99/ed35197a158f1fdc2fe7c3680


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Import Libraries

In [3]:
# Install missing packages in the notebook kernel (required to resolve ModuleNotFoundError)
import cv2
import mediapipe as mp
import numpy as np
import time
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os

print("Libraries imported successfully!")
print(f"MediaPipe version: {mp.__version__}")
print(f"OpenCV version: {cv2.__version__}")

Libraries imported successfully!
MediaPipe version: 0.10.21
OpenCV version: 4.11.0


## 3. Helper Functions for Angle Calculation

In [6]:
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    Args:
        a, b, c: Landmark points (x, y coordinates)
    Returns:
        angle: Angle in degrees
    """
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

def extract_pose_features(landmarks):
    """
    Extract relevant features from pose landmarks for situp detection
    """
    # Get key landmarks
    left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]
    
    right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_hip = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].y]
    
    # Calculate angles
    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    
    # Average hip angle
    avg_hip_angle = (left_hip_angle + right_hip_angle) / 2
    
    # Torso vertical position (y-coordinate of shoulders)
    torso_position = (left_shoulder[1] + right_shoulder[1]) / 2
    
    return [avg_hip_angle, torso_position, left_hip[1], right_hip[1]]

print("Helper functions defined successfully!")

Helper functions defined successfully!


## 4. Situp Counter Class

In [7]:
class SitupCounter:
    def __init__(self, angle_threshold_down=60, angle_threshold_up=100):
        """
        Initialize situp counter
        Args:
            angle_threshold_down: Hip angle threshold for down position
            angle_threshold_up: Hip angle threshold for up position
        """
        self.counter = 0
        self.stage = None  # 'down' or 'up'
        self.angle_threshold_down = angle_threshold_down
        self.angle_threshold_up = angle_threshold_up
        
    def update(self, hip_angle):
        """
        Update counter based on hip angle
        """
        # Down position (lying down)
        if hip_angle > self.angle_threshold_up:
            self.stage = "down"
        
        # Up position (sitting up)
        if hip_angle < self.angle_threshold_down and self.stage == "down":
            self.stage = "up"
            self.counter += 1
            
        return self.counter, self.stage
    
    def reset(self):
        """Reset counter"""
        self.counter = 0
        self.stage = None

print("SitupCounter class created successfully!")

SitupCounter class created successfully!


## 5. Generate Training Data

We'll generate synthetic training data to create a classifier that can distinguish between different situp positions.

In [8]:
def generate_training_data():
    """
    Generate synthetic training data for situp position classification
    Returns:
        X: Feature array [hip_angle, torso_position, left_hip_y, right_hip_y]
        y: Labels (0=down, 1=up, 2=transition)
    """
    np.random.seed(42)
    
    X = []
    y = []
    
    # Down position (lying down) - hip angle > 100 degrees
    for _ in range(300):
        hip_angle = np.random.uniform(100, 170)
        torso_position = np.random.uniform(0.6, 0.8)  # Lower on screen
        left_hip_y = np.random.uniform(0.6, 0.8)
        right_hip_y = np.random.uniform(0.6, 0.8)
        X.append([hip_angle, torso_position, left_hip_y, right_hip_y])
        y.append(0)  # Down position
    
    # Up position (sitting up) - hip angle < 60 degrees
    for _ in range(300):
        hip_angle = np.random.uniform(30, 60)
        torso_position = np.random.uniform(0.3, 0.5)  # Higher on screen
        left_hip_y = np.random.uniform(0.4, 0.6)
        right_hip_y = np.random.uniform(0.4, 0.6)
        X.append([hip_angle, torso_position, left_hip_y, right_hip_y])
        y.append(1)  # Up position
    
    # Transition position
    for _ in range(200):
        hip_angle = np.random.uniform(60, 100)
        torso_position = np.random.uniform(0.4, 0.7)
        left_hip_y = np.random.uniform(0.5, 0.7)
        right_hip_y = np.random.uniform(0.5, 0.7)
        X.append([hip_angle, torso_position, left_hip_y, right_hip_y])
        y.append(2)  # Transition
    
    return np.array(X), np.array(y)

# Generate data
X, y = generate_training_data()
print(f"Generated {len(X)} training samples")
print(f"Feature shape: {X.shape}")
print(f"Label distribution: Down={np.sum(y==0)}, Up={np.sum(y==1)}, Transition={np.sum(y==2)}")

Generated 800 training samples
Feature shape: (800, 4)
Label distribution: Down=300, Up=300, Transition=200


## 6. Train the Model

In [9]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
model.fit(X_train, y_train)

# Evaluate model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Training accuracy: {train_score:.3f}")
print(f"Testing accuracy: {test_score:.3f}")
print("\nModel trained successfully!")

Training accuracy: 1.000
Testing accuracy: 1.000

Model trained successfully!


## 7. Save the Model

In [10]:
# Create Models directory if it doesn't exist
models_dir = "../Models"
os.makedirs(models_dir, exist_ok=True)

# Save the model
model_path = os.path.join(models_dir, "situp_counter_model.pkl")
joblib.dump(model, model_path)

print(f"Model saved successfully to: {model_path}")
print(f"Model file size: {os.path.getsize(model_path) / 1024:.2f} KB")

Model saved successfully to: ../Models\situp_counter_model.pkl
Model file size: 131.14 KB


## 8. Real-time Situp Counter (Using Webcam)

This function demonstrates how to use the model with a webcam for real-time situp counting over 30 seconds.

In [12]:
def count_situps_realtime(duration=30):
    """
    Count situps in real-time using webcam
    Args:
        duration: Time duration in seconds (default 30)
    """
    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    
    # Initialize situp counter
    situp_counter = SitupCounter()
    
    # Load the trained model
    model_path = "../Models/situp_counter_model.pkl"
    trained_model = joblib.load(model_path)
    
    # Setup video capture
    cap = cv2.VideoCapture(0)
    
    # Get start time
    start_time = time.time()
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Calculate elapsed time
            elapsed_time = time.time() - start_time
            remaining_time = max(0, duration - elapsed_time)
            
            # Stop after duration
            if elapsed_time > duration:
                break
            
            # Convert to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            # Make detection
            results = pose.process(image)
            
            # Convert back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Process if landmarks detected
            if results.pose_landmarks:
                # Extract features
                landmarks = results.pose_landmarks.landmark
                features = extract_pose_features(landmarks)
                hip_angle = features[0]
                
                # Update counter
                count, stage = situp_counter.update(hip_angle)
                
                # Draw landmarks
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                )
                
                # Display info on frame
                cv2.rectangle(image, (0,0), (300,120), (245,117,16), -1)
                
                cv2.putText(image, f'COUNT: {count}', (10,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
                cv2.putText(image, f'STAGE: {stage}', (10,60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
                cv2.putText(image, f'TIME: {int(remaining_time)}s', (10,90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
            
            # Display frame
            cv2.imshow('Situp Counter', image)
            
            # Exit on 'q' key
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"\n{'='*50}")
    print(f"SESSION COMPLETE!")
    print(f"Total Situps: {situp_counter.counter}")
    print(f"Duration: {duration} seconds")
    print(f"{'='*50}")
    
    return situp_counter.counter

print("Real-time situp counter function ready!")
print("\nTo start counting, run: count_situps_realtime(30)")

Real-time situp counter function ready!

To start counting, run: count_situps_realtime(30)


## 9. Test Model Loading

Verify that the saved model can be loaded correctly.

In [13]:
# Load the model
loaded_model = joblib.load("../Models/situp_counter_model.pkl")

# Test prediction with sample data
test_sample = np.array([[150, 0.7, 0.7, 0.7]])  # Down position
prediction = loaded_model.predict(test_sample)
position_map = {0: 'Down', 1: 'Up', 2: 'Transition'}

print(f"Model loaded successfully!")
print(f"Test sample features: {test_sample[0]}")
print(f"Predicted position: {position_map[prediction[0]]}")
print(f"\nThe model is ready to use!")

Model loaded successfully!
Test sample features: [150.    0.7   0.7   0.7]
Predicted position: Down

The model is ready to use!


## 10. Usage Instructions

### To count situps in real-time:
```python
# Run for 30 seconds (default)
count_situps_realtime(30)

# Or run for custom duration
count_situps_realtime(60)  # 60 seconds
```

### Key Points:
- Position yourself so your full body is visible in the webcam
- Lie down with knees bent for the starting position
- The system tracks hip angles to detect situp movements
- Press 'q' to exit early
- The counter will automatically stop after the specified duration

### Model Information:
- **Location**: `../Models/situp_counter_model.pkl`
- **Type**: Random Forest Classifier
- **Features**: Hip angle, torso position, hip coordinates
- **Classes**: Down (0), Up (1), Transition (2)